In [1]:
from pathlib import Path
import pandas as pd
import scanpy as sc
from tqdm import tqdm

In [2]:
data_dir = Path('/Users/djemec/data/jepa/v0_3')
raw_data = data_dir / 'raw_data'

In [3]:
#source: downloaded from https://plus.figshare.com/ndownloader/files/35773070
# https://gwps.wi.mit.edu/data_download

raw_h5 = raw_data / 'K562_essential_raw_singlecell_01.h5ad'
crispr_lib_v2 = raw_data / 'hcrispri_v2.csv'
crispr_lib_v2_1 = raw_data / 'hcrispri_v2_1.csv'

In [72]:
adata = sc.read_h5ad(raw_h5)
adata.obs.columns

Index(['gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript',
       'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI',
       'core_scale_factor', 'core_adjusted_UMI_count'],
      dtype='object')

In [73]:
v2_lib_df = pd.read_csv(crispr_lib_v2)
v2_lib_df.head()

,sgID,gene,transcript,protospacer sequence,selection rank,predicted score,empirical score,off-target stringency,Sublibrary half
0,A1BG_-_58858617.23-P1,A1BG,P1,GGAGACCCAGCGCTAACCAG,1.0,1.008816,NaN,0,Top5
1,A1BG_-_58858788.23-P1,A1BG,P1,GGGGCACCCAGGAGCGGTAG,2.0,0.901176,NaN,0,Top5
2,A1BG_+_58858964.23-P1,A1BG,P1,GCTCCGGGCGACGTGGAGTG,3.0,0.836188,NaN,0,Top5
3,A1BG_-_58858630.23-P1,A1BG,P1,GAACCAGGGGTGCCCAAGGG,4.0,0.827551,NaN,0,Top5
4,A1BG_+_58858549.23-P1,A1BG,P1,GGCGAGGAACCGCCCAGCAA,5.0,0.775395,NaN,0,Top5


In [74]:
v21_lib_df = pd.read_csv(crispr_lib_v2_1)
v21_lib_df.head()

,sgID,gene,transcript,protospacer sequence,selection rank,predicted score,empirical score,off-target stringency,Sublibrary half
0,A1BG_+_58858964.23-P1,A1BG,P1,GCTCCGGGCGACGTGGAGTG,1,0.870837,NaN,0,Top5
1,A1BG_-_58858788.23-P1,A1BG,P1,GGGGCACCCAGGAGCGGTAG,2,0.782793,NaN,0,Top5
2,A1BG_-_58858991.23-P1,A1BG,P1,GTCCACGTCGCCCGGAGCTG,3,0.722963,NaN,0,Top5
3,A1BG_-_58858950.23-P1,A1BG,P1,GGCAGCGCAGGACGGCATCT,4,0.706643,NaN,0,Top5
4,A1BG_-_58858915.23-P1,A1BG,P1,GAGCAGCTCGAAGGTGACGT,5,0.700359,NaN,0,Top5


In [75]:
id_to_seq_2 = dict(zip(v2_lib_df['sgID'].str.strip(), v2_lib_df['protospacer sequence'].str.strip()))
id_to_seq_21 = dict(zip(v21_lib_df['sgID'].str.strip(), v21_lib_df['protospacer sequence'].str.strip()))

In [76]:
def get_sequences(dual_id_string):
    '''
    Parses 'GuideA_ID|GuideB_ID' and returns 'SeqA|SeqB'
    '''
    if pd.isna(dual_id_string) or dual_id_string == '':
        return None, None
        
    try:
        # The IDs in sgID_AB are separated by a pipe '|'
        parts = dual_id_string.split('|')
        
        if len(parts) == 2:
            id_a, id_b = parts
            
            # Look up sequences (Default to 'NOT_FOUND' if missing)
            seq_a = id_to_seq_2.get(id_a.strip(), id_to_seq_21.get(id_a.strip(), 'NOT_FOUND'))
            seq_b = id_to_seq_2.get(id_b.strip(), id_to_seq_21.get(id_b.strip(), 'NOT_FOUND'))
            
            return seq_a, seq_b
            
        elif len(parts) == 1:
            # Handle rare single-guide cases if they exist
            seq_a = id_to_seq_2.get(parts[0].strip(), id_to_seq_21.get(parts[0].strip(), 'NOT_FOUND'))
            return seq_a, 'NONE'
            
    except Exception as e:
        return 'ERROR', 'ERROR'

    return 'MISSING', 'MISSING'

In [77]:
dual_ids = adata.obs['sgID_AB'].astype(str).tolist()
len(list(set(adata.obs['sgID_AB'].astype(str).tolist())))

2273

In [78]:
seq_a_list = []
seq_b_list = []

In [79]:
for dual_id in tqdm(dual_ids):
    sa, sb = get_sequences(dual_id)
    seq_a_list.append(sa)
    seq_b_list.append(sb)

100%|██████████████████████████████████████████████████████████████| 310385/310385 [00:00<00:00, 2192558.33it/s]


In [80]:
adata.obs['guide_seq_a'] = seq_a_list
adata.obs['guide_seq_b'] = seq_b_list

In [81]:
print(adata.obs[['sgID_AB', 'guide_seq_a', 'guide_seq_b']].head(5))

                                                               sgID_AB  \
cell_barcode                                                             
AAACCCAAGAAATCCA-27  NAF1_+_164087918.23-P1P2|NAF1_-_164087674.23-P1P2   
AAACCCAAGAACTTCC-31  BUB1_-_111435363.23-P1P2|BUB1_-_111435372.23-P1P2   
AAACCCAAGAAGCCAC-34      UBL5_-_9938639.23-P1P2|UBL5_+_9938801.23-P1P2   
AAACCCAAGAATAGTC-43  C9orf16_+_130922603.23-P1P2|C9orf16_+_13092264...   
AAACCCAAGACAGCGT-28  TIMM9_-_58893843.23-P1P2|TIMM9_-_58893848.23-P1P2   

                              guide_seq_a           guide_seq_b  
cell_barcode                                                     
AAACCCAAGAAATCCA-27  GGAGCCGTGAGCTTGTCCAG  GCCGCGACGGCGTTCAGAAC  
AAACCCAAGAACTTCC-31  GGACAAGCGCCGGGCCTCAG  GCGGGCCTCAGCGGAACCCA  
AAACCCAAGAAGCCAC-34  GGGTGAGGAGCTGGTGGCGT  GCCCAGGGCCGCGAACCCCG  
AAACCCAAGAATAGTC-43  GGCCGGCGCCGGATGGAAGG  GGCCGCGCGACGATGGAACG  
AAACCCAAGACAGCGT-28  GGGGACGGTTGAGCCTTGGG  GGGTTGAGCCTTGGGAGGGA  


In [82]:
len(set(adata.obs[adata.obs['guide_seq_a'] == 'NOT_FOUND']['sgID_AB'].astype(str)))

9

In [83]:
len(set(adata.obs[adata.obs['guide_seq_b'] == 'NOT_FOUND']['sgID_AB'].astype(str)))

9

In [84]:
set(adata.obs[adata.obs['guide_seq_a'] == 'NOT_FOUND']['sgID_AB'])

{'BRD4_-_15391173.23-ENST00000263377.2-ENST00000371835.4|BRD4_-_15391126.23-ENST00000263377.2-ENST00000371835.4',
 'C7orf50_+_1177904.23-ENST00000357429.6-ENST00000397098.3-ENST00000397100.2|C7orf50_+_1177839.23-ENST00000357429.6-ENST00000397098.3-ENST00000397100.2',
 'EIF3CL_+_28415115.23-ENST00000380876.4-ENST00000398943.3-ENST00000398944.3|EIF3CL_-_28415135.23-ENST00000380876.4-ENST00000398943.3-ENST00000398944.3',
 'FRG2_-_190948162.23-ENST00000378763.1-ENST00000504750.1|FRG2_-_190948402.23-ENST00000378763.1-ENST00000504750.1',
 'GTF2H2_+_70363453.23-ENST00000274400.5-ENST00000330280.7|GTF2H2_+_70363440.23-ENST00000274400.5-ENST00000330280.7',
 'SPDYE2_+_102191744.23-ENST00000432940.1-ENST00000507918.1|SPDYE2_+_102191690.23-ENST00000432940.1-ENST00000507918.1',
 'TRIM49C_-_89764362.23-ENST00000432771.1-ENST00000448984.1|TRIM49C_+_89764400.23-ENST00000432771.1-ENST00000448984.1',
 'ZNF658_+_40791787.23-ENST00000377626.3-ENST00000602553.1|ZNF658_+_40791781.23-ENST00000377626.3-ENST00